In [2]:
!pip install torchtext

In [3]:
#https://drive.google.com/file/d/1o09HEkDBBEjsW3gN76vDYfCAKiCqpcpg/view
!gdown --id 1o09HEkDBBEjsW3gN76vDYfCAKiCqpcpg

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1o09HEkDBBEjsW3gN76vDYfCAKiCqpcpg
To: /content/financial_sentiment.zip
100% 925k/925k [00:00<00:00, 10.4MB/s]


In [4]:
!unzip '/content/financial_sentiment.zip'

Archive:  /content/financial_sentiment.zip
  inflating: FinancialPhraseBank/License.txt  
  inflating: FinancialPhraseBank/README.txt  
  inflating: FinancialPhraseBank/Sentences_50Agree.txt  
  inflating: FinancialPhraseBank/Sentences_66Agree.txt  
  inflating: FinancialPhraseBank/Sentences_75Agree.txt  
  inflating: FinancialPhraseBank/Sentences_AllAgree.txt  
  inflating: all-data.csv            


In [5]:
import torch
import torch.nn as nn
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F
import nltk
import re #Thư viện xử lý Regular Expressions (biểu thức chính quy) để tìm kiếm và thao tác văn bản.
import unicodedata
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
torch.manual_seed(1)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
df = pd.read_csv('/content/all-data.csv', names=['sentiment','content'], encoding='ISO-8859-1')
df.head(10)



,sentiment,content
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
5,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
6,positive,"For the last quarter of 2010 , Componenta 's n..."
7,positive,"In the third quarter of 2010 , net sales incre..."
8,positive,Operating profit rose to EUR 13.1 mn from EUR ...
9,positive,"Operating profit totalled EUR 21.1 mn , up fro..."


In [7]:
classes = df['sentiment'].unique()
classes = {class_name : idx for idx,class_name in enumerate(classes.tolist()) }
df['sentiment'] = df['sentiment'].map(classes)
df.head(10)

,sentiment,content
0,0,"According to Gran , the company has no plans t..."
1,0,Technopolis plans to develop in stages an area...
2,1,The international electronic industry company ...
3,2,With the new production plant the company woul...
4,2,According to the company 's updated strategy f...
5,2,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
6,2,"For the last quarter of 2010 , Componenta 's n..."
7,2,"In the third quarter of 2010 , net sales incre..."
8,2,Operating profit rose to EUR 13.1 mn from EUR ...
9,2,"Operating profit totalled EUR 21.1 mn , up fro..."


In [8]:
#data procesing
eng_stop_words = stopwords.words('english') #eg : is,the,in
stemmer = PorterStemmer() #eg : running ->run
def test_nomalize(text):
    text = text.lower().strip()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in eng_stop_words])
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

In [9]:
for i in df['content'].index:
    df['content'][i] = test_nomalize(df['content'][i])
df.head(10)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['content'][i] = test_nomalize(df['content'][i])
<ipython-input-9-4e8917b00378>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'][i] = test_nomalize(df['content'][i])
<ipython-input-9-4e8917b00378>:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a co

,sentiment,content
0,0,accord gran compani plan move product russia a...
1,0,technopoli plan develop stage area less 100000...
2,1,intern electron industri compani elcoteq laid ...
3,2,new product plant compani would increas capac ...
4,2,accord compani updat strategi year 20092012 ba...
5,2,financ aspocomp growth aspocomp aggress pursu ...
6,2,last quarter 2010 componenta net sale doubl eu...
7,2,third quarter 2010 net sale increas 52 eur 205...
8,2,oper profit rose eur 131 mn eur 87 mn correspo...
9,2,oper profit total eur 211 mn eur 186 mn 2007 r...


In [10]:
vocab = []
for sentence in df['content']:
    for word in sentence.split():
        if word not in vocab:
            vocab.append(word)
vocab.append('<UNK>')
vocab.append('<PAD>')
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
print(len(vocab))

8905


In [21]:
def transforms(text, word_to_idx, max_seq_len):
    tokens = []
    # Add a check for unknown words
    for word in text.split():
        if word in word_to_idx:
            tokens.append(word_to_idx[word])
        else:
            # Handle unknown words (e.g., replace with an 'UNK' token)
            if 'UNK' in word_to_idx:  # Check if 'UNK' token exists
                tokens.append(word_to_idx['UNK'])
            else:
                # If 'UNK' token is not defined, you might want to skip the word or handle it differently.
                # Here, I'm skipping the unknown word for simplicity.
                pass

    # Padding
    if len(tokens) < max_seq_len:
        # Check if 'PAD' token exists, otherwise use 0 as padding
        pad_token = word_to_idx.get('PAD', 0)
        tokens += [pad_token] * (max_seq_len - len(tokens))
    elif len(tokens) > max_seq_len:
        tokens = tokens[:max_seq_len]
    return tokens

In [22]:
X_train,X_val,y_train,y_val = train_test_split(
    df['content'].tolist(),df['sentiment'].tolist(),
    test_size=0.2,random_state=42,shuffle=True)
X_tran,X_test,y_train,y_test=train_test_split(
    X_train,y_train,test_size=0.125,random_state=42,shuffle=True)
len(X_tran),len(X_val),len(X_test)

(3391, 970, 485)

In [23]:
class FinancialNews(Dataset):
    def __init__(self,X,y,word_to_idx,max_seq_len,transforms=None):
        self.X = X
        self.y = y
        self.transforms = transforms
        self.word_to_idx = word_to_idx
        self.max_seq_len = max_seq_len
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        text = self.X[idx]
        label = self.y[idx]
        if self.transforms:
            text = self.transforms(text,self.word_to_idx,self.max_seq_len)
            return torch.tensor(text),torch.tensor(label)

In [24]:
train_dataset = FinancialNews(X_tran,y_train,word_to_idx,max_seq_len=40,transforms=transforms)
val_dataset = FinancialNews(X_val,y_val,word_to_idx,max_seq_len=40,transforms=transforms)
test_dataset = FinancialNews(X_test,y_test,word_to_idx,max_seq_len=40,transforms=transforms)

train_loader = DataLoader(train_dataset,batch_size=132,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=8,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=8,shuffle=True)
train_loader,val_loader,test_loader

(<torch.utils.data.dataloader.DataLoader at 0x7b66f5a44c70>,
 <torch.utils.data.dataloader.DataLoader at 0x7b66f5a44bb0>)

In [25]:
class SentimentClassifier(nn.Module):
    def __init__(self,vocab_size,embed_dim,hidden_dim,n_layers,n_classes,dropout):
        super(SentimentClassifier,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embed_dim)
        self.rnn = nn.RNN(embed_dim,hidden_dim,n_layers,batch_first=True,dropout=dropout)
        self.norm=nn.LayerNorm(hidden_dim)
        self.dropout=nn.Dropout(dropout)
        self.fc1 =nn.Linear(hidden_dim,16)
        self.relu=nn.ReLU()
        self.fc2 =nn.Linear(16,n_classes)

    def forward(self,x):
        x = self.embedding(x)
        x,h = self.rnn(x)
        x = self.norm(x[:,-1])
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [28]:
n_classes = len(classes)
embed_dim = 128
hidden_dim = 128
n_layers = 2
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentimentClassifier(len(vocab),embed_dim,hidden_dim,n_layers,n_classes,dropout).to(device)
print(model)
print(model.parameters())

SentimentClassifier(
  (embedding): Embedding(8905, 128)
  (rnn): RNN(128, 128, num_layers=2, batch_first=True, dropout=0.2)
  (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=128, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=3, bias=True)
)
<generator object Module.parameters at 0x7b66f4ba3760>


In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [30]:
def fit(model,train_loader,val_loader,criterion,optimizer,device,epochs):
  train_losses = []
  val_losses = []
  for epoch in range(epochs):
      batch_train_losses=[]
      model.train()
      for idx, (data,target) in enumerate(train_loader):
          data,target = data.to(device),target.to(device)

          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output,target)
          loss.backward()
          optimizer.step()
          batch_train_losses.append(loss.item())

      train_loss = np.mean(batch_train_losses)
      train_losses.append(train_loss)

      val_loss,val_acc = evaluate(model,val_loader,criterion,device)
      val_losses.append(val_loss)
      print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}')

  return train_losses,val_losses



In [31]:
def evaluate(model,val_loader,criterion,device):
  model.eval()
  correct=0
  tottal=0
  batch_val_losses=[]
  with torch.no_grad():
      for idx, (data,target) in enumerate(val_loader):
          data,target = data.to(device),target.to(device)
          output = model(data)
          loss = criterion(output,target)
          batch_val_losses.append(loss.item())
          pred = output.argmax(dim=1,keepdim=True)
          correct += pred.eq(target.view_as(pred)).sum().item()
          tottal += data.size(0)

  val_loss = np.mean(batch_val_losses)
  val_acc = correct/tottal
  return val_loss,val_acc


In [33]:
train_losses,val_losses = fit(model,train_loader,val_loader,criterion,optimizer,device,epochs=1000)


Epoch 1/1000 | Train Loss: 0.9281 | Val Loss: 0.9195 | Val Accuracy: 0.5887
Epoch 2/1000 | Train Loss: 0.9278 | Val Loss: 0.9211 | Val Accuracy: 0.5887
Epoch 3/1000 | Train Loss: 0.9273 | Val Loss: 0.9239 | Val Accuracy: 0.5887
Epoch 4/1000 | Train Loss: 0.9274 | Val Loss: 0.9194 | Val Accuracy: 0.5887
Epoch 5/1000 | Train Loss: 0.9274 | Val Loss: 0.9191 | Val Accuracy: 0.5887
Epoch 6/1000 | Train Loss: 0.9277 | Val Loss: 0.9212 | Val Accuracy: 0.5887
Epoch 7/1000 | Train Loss: 0.9276 | Val Loss: 0.9186 | Val Accuracy: 0.5887
Epoch 8/1000 | Train Loss: 0.9293 | Val Loss: 0.9213 | Val Accuracy: 0.5887
Epoch 9/1000 | Train Loss: 0.9286 | Val Loss: 0.9213 | Val Accuracy: 0.5887
Epoch 10/1000 | Train Loss: 0.9274 | Val Loss: 0.9214 | Val Accuracy: 0.5887
Epoch 11/1000 | Train Loss: 0.9289 | Val Loss: 0.9257 | Val Accuracy: 0.5887
Epoch 12/1000 | Train Loss: 0.9273 | Val Loss: 0.9215 | Val Accuracy: 0.5887
Epoch 13/1000 | Train Loss: 0.9282 | Val Loss: 0.9218 | Val Accuracy: 0.5887
Epoch 14

In [34]:
val_loss,val_acc = evaluate(model,val_loader,criterion,device)
print(f'Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}')

Val Loss: 0.9215 | Val Accuracy: 0.5887
